<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: Summarization Tests

**Description:** This assignment covers summarization outputs. You will compare three different types of solutions, all using an encoder-decoder architecture. You should also be able to develop an intuition for:


* How well summarization systems work
* The effects of using different pre-training and fine-tuning checkpoints on outcomes
* The effects of hyperparameters on outcomes
* Evaluation of output using ROUGE



This notebook should be run on a Google Colab but it does not require a GPU. By default, when you open the notebook in Colab it will NOT configure a GPU.  Summarization commands can take up to five minutes to run depending on the hyperparameters you use. This notebook will NOT run on your GCP instance as the summary models are larger than the avaialble memory.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2023-summer-main/blob/master/assignment/a3/Summarization_test.ipynb)

The overall assignment structure is as follows:

 Setup

1. T5 for generic summarization

2. Pegasus for headline summarization

3. Pegasus for longer generation




**INSTRUCTIONS:**:

* Questions are always indicated as **QUESTION:**, so you can search for this string to make sure you answered all of the questions. You are expected to fill out, run, and submit this notebook, as well as to answer the questions in the **answers** file as you did in a1 and a2.

* **### YOUR CODE HERE** indicates that you are supposed to write code.




## Setup

In [1]:
!pip install -q sentencepiece

In [2]:
!pip install -q transformers

In [3]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you 

In [4]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [5]:
#let's make longer output readable without horizontal scrolling
from pprint import pprint

Let's leverage the pre-trained and fine tuned models on HuggingFace to demonstrate some capabilities with abstractive summarization and language generation.  They include models/checkpoints that were fine tuned on a particular dataset.  In our case we'll focus on one dataset that emphasizes a one line output and another that emphasizes a multi-line output.

We'll use this same toy article as the input to all of our summarization attempts.  That way we have the ability to compare. We'll also create two references for evaluation.  These are the targets you are trying to meet.  One reference is for the longer output. The second reference is the short one for the one line output.

In [ ]:

ARTICLE_TO_SUMMARIZE = (
    "Nearly 800 thousand customers are scheduled to be affected by the shutoffs which are expected to last through at least midday tomorrow. "
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. "
    "The aim is to reduce the risk of wildfires. "
    "If Pacific Gas & Electric Co, a unit of PG&E Corp, goes through with another public safety power shutoff, "
    " it would be the fourth round of mass blackouts imposed by the utility since Oct. 9, when some 730,000 customers were left in the dark. "
    "The recent wave of precautionary shutoffs have drawn sharp criticism from Governor Gavin Newsom, state regulators and consumer activists as being overly broad in scale."
    "Newsom blames PG&E for doing too little to properly maintain and secure its power lines against wind damage."
    "Utility executives have acknowledged room for improvement while defending the sprawling scope of the power cutoffs as a matter of public safety."
    "The record breaking drought has made the current conditions even worse than in previous years. "
    "It exponentially increases the probability of large scale wildfires. "
)

LONG_REFERENCE = (
    "Many PG&E customers could be affected by public safety power shutoffs in response to forecasts for high winds and dry conditions. "
    "The record breaking drought exponentially increases the probability of large scale wildfires. "
    "Despite being criticized by Governor Newsom for being overly broad, company officials defend the cutoffs as a matter of public safety. "
)

SHORT_REFERENCE = (
    "California's largest utility is set to turn off power to hundreds of thousands of customers in an effort to reduce the risk of wildfires. "
)

How long is our article to summarize?  Obviously our summary should be shorter since it is supposed to be "abridged."

In [ ]:
len(str.split(ARTICLE_TO_SUMMARIZE))

177

In [6]:
# Install Necessary Libraries (if not already installed)
!pip install datasets pandas


In [7]:
# Load the Dataset
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("xsum")

# Convert to Pandas DataFrames for Easier Manipulation
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [8]:
# Explore the Dataset
print("Number of Training Examples:", len(train_df))
print("Number of Validation Examples:", len(validation_df))
print("Number of Test Examples:", len(test_df))
print(train_df.head().to_markdown(index=False, numalign="left", stralign="left")) # Show first 5 rows of the training set in a markdown table


Number of Training Examples: 204045
Number of Validation Examples: 11332
Number of Test Examples: 11334
| document                                                                                                                                                                                                                                                                            | summary                                                                                                                                       | id       |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|:---------|
| The full cost of damage 

## 1. T5 for Generic Summarization

T5 is an encoder decoder architecture that has been trained on multiple tasks, so not purely summarization.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/t5).

In [9]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
t5model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Since T5 can perform multiple tasks we need to tell it what kind of output we want.  Therefore we need to prepend a "prompt" to our article text to make sure it does the right thing.

In [11]:
from datasets import load_dataset
from transformers import pipeline
from rouge_score import rouge_scorer

# Load and sample a subset of XSUM (e.g., 500 examples)
dataset = load_dataset("xsum")
sample_size = 1
sampled_dataset = dataset['train'].shuffle(seed=42).select(range(sample_size))

# Load the T5 summarization model
summarizer = pipeline("summarization", model=t5model, tokenizer=t5tokenizer)


# Summarize the subset
generated_summaries = []
for example in sampled_dataset:
    summary = summarizer(example['document'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)[0]['summary_text']
    generated_summaries.append(summary)

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for ref_summary, gen_summary in zip(sampled_dataset['summary'], generated_summaries):
    scores = scorer.score(ref_summary, gen_summary)
    rouge_scores.append(scores)

# Aggregate and display scores
avg_scores = {
    metric: sum(score[metric].fmeasure for score in rouge_scores) / len(rouge_scores)
    for metric in ['rouge1', 'rouge2', 'rougeL']
}
print("Average ROUGE scores:", avg_scores)


Average ROUGE scores: {'rouge1': 0.1492537313432836, 'rouge2': 0.0, 'rougeL': 0.11940298507462685}


Let's just run T5 using it's default hyperparameters and see what happens.  We'll hold on to the output in the candidate variable.  What do you think about the output?

In [12]:
#In order to not consume all of the memory available in Colab we'll free up the memory we're using for these large language models
del t5model
del t5tokenizer


## 2. Pegasus for Headline Summarization

Pegasus is an encoder decoder architecture that has been explicitly pre-trained as an abstractive summarizer.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/pegasus) and [here](https://arxiv.org/pdf/1912.08777.pdf).

We'll first use the `google/pegasus-xsum` checkpoint.  It is trained on a [summarization task](https://aclanthology.org/D18-1206.pdf) that reads a news article and then [emits a one line summary](https://huggingface.co/datasets/xsum).  This doesn't mean that it is limited in its output length.  It does mean that it works well with news article type inputs and tends toward shorter outputs.

In [13]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [14]:
pmodel.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
 final_logits_bias (BiasLay  multiple                  96103     
 er)                                                             
                                                                 
Total params: 569844583 (2.12 GB)
Trainable params: 569748480 (2.12 GB)
Non-trainable params: 96103 (375.40 KB)
_________________________________________________________________


In [15]:
# Load the T5 summarization model
summarizer = pipeline("summarization", model=pmodel, tokenizer=ptokenizer)


# Summarize the subset
generated_summaries = []
for example in sampled_dataset:
    summary = summarizer(example['document'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)[0]['summary_text']
    generated_summaries.append(summary)

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for ref_summary, gen_summary in zip(sampled_dataset['summary'], generated_summaries):
    scores = scorer.score(ref_summary, gen_summary)
    rouge_scores.append(scores)

# Aggregate and display scores
avg_scores = {
    metric: sum(score[metric].fmeasure for score in rouge_scores) / len(rouge_scores)
    for metric in ['rouge1', 'rouge2', 'rougeL']
}
print("Average ROUGE scores:", avg_scores)

Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors


Average ROUGE scores: {'rouge1': 0.2692307692307692, 'rouge2': 0.12, 'rougeL': 0.2692307692307692}
